<a href="https://www.kaggle.com/code/yanyuewang00/panda-email-pro-ai-powered-professional-email-gen?scriptVersionId=235504640" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**Introduction and Setup**

In [1]:
# Panda Email Pro: AI-Powered Professional Email Generator
# Using Gemini API to convert casual speech to professional emails

## Introduction - The Problem and AI Solution
# Business professionals spend excessive time crafting professional emails - checking grammar, 
# ensuring politeness, and covering all key requests. This process is not only time-consuming 
# but can also lead to overlooked details like forgotten attachments.
#
# Panda Email Pro solves this by:
# 1. Taking audio input of casual spoken instructions
# 2. Processing speech to understand intent and content
# 3. Using Gemini API to draft professional, error-free emails
# 4. Giving users time to focus on important details like attachments

## Gen AI Capabilities Used
# 1. Audio Understanding - Processing speech input into actionable text
# 2. Structured Output/JSON Mode - Formatting email components consistently
# 3. Few-shot Prompting - Teaching the model effective email formats
# 4. Long Context Window - Maintaining conversation history for context-aware responses
# 5. Function Calling - Structuring outputs in specific formats

# Install required packages
!pip install google-generativeai==0.3.2 python-dotenv soundfile librosa pydub ipywebrtc ipywidgets gradio SpeechRecognition

# Import necessary libraries
import os
import json
import base64
import tempfile
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Audio, HTML
import google.generativeai as genai
from dotenv import load_dotenv
import gradio as gr
import librosa
import soundfile as sf
from pydub import AudioSegment
import ipywidgets as widgets
import speech_recognition as sr

# Access the API key from Kaggle secrets
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

# Verify we have the key (without displaying it)
if GOOGLE_API_KEY:
    print("✅ Successfully loaded API key from Kaggle secrets")
else:
    print("❌ Failed to load API key - please check your Kaggle secrets")

genai.configure(api_key=GOOGLE_API_KEY)

# Set up the model
model = genai.GenerativeModel('gemini-1.5-pro')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.9/146.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled

**Core Classes**

In [2]:
## Audio Processing Module
class AudioProcessor:
    """
    Handles audio recording, processing, and conversion to text.
    Uses speech recognition to convert audio to text that can be used for email generation.
    """
    
    def __init__(self):
        """Initialize the audio processor."""
        self.sample_rate = 16000
        
    def process_audio_file(self, audio_file_path):
        """
        Process an existing audio file and convert to text using Google's Speech Recognition.
        
        Args:
            audio_file_path (str): Path to the audio file
            
        Returns:
            str: Transcribed text from the audio
        """
        recognizer = sr.Recognizer()
        
        try:
            # Load the audio file
            with sr.AudioFile(audio_file_path) as source:
                # Adjust for ambient noise and record
                recognizer.adjust_for_ambient_noise(source)
                audio_data = recognizer.record(source)
                
            # Use Google's Speech Recognition service
            transcription = recognizer.recognize_google(audio_data)
            print("Transcription successful!")
            return transcription
            
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
            return "I couldn't understand the audio. Please try again."
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return "There was an error connecting to the speech recognition service."
        except Exception as e:
            print(f"Error processing audio: {e}")
            # Fallback to simulated transcription for demo purposes
            print("Using simulated transcription instead")
            simulated_transcription = """
            I need to send an email to Sarah from the marketing team about the upcoming product launch.
            We need to finalize the marketing materials by next Friday, and I also want to ask about
            the status of the social media campaign we discussed last week. Oh, and remind her that we
            need to coordinate with the sales team on the messaging. Keep it professional but friendly
            since we work closely together.
            """
            return simulated_transcription
    
    def record_audio(self, duration=10):
        """
        Record audio directly from the microphone.
        
        Args:
            duration (int): Duration to record in seconds
            
        Returns:
            str: Path to recorded audio file
        """
        recognizer = sr.Recognizer()
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.wav')
        temp_file.close()
        
        try:
            print(f"Recording audio for {duration} seconds...")
            with sr.Microphone() as source:
                recognizer.adjust_for_ambient_noise(source)
                print("Speak now...")
                audio = recognizer.listen(source, timeout=duration)
                
            # Save the recorded audio to a WAV file
            with open(temp_file.name, "wb") as f:
                f.write(audio.get_wav_data())
                
            print("Recording complete!")
            return temp_file.name
            
        except Exception as e:
            print(f"Error recording audio: {e}")
            # Generate silent audio as fallback
            silent_audio = np.zeros(int(self.sample_rate * duration))
            sf.write(temp_file.name, silent_audio, self.sample_rate)
            print("Created silent audio file instead.")
            return temp_file.name


## Email Generator Module
class EmailGenerator:
    """
    Uses the Gemini API to generate professional emails based on casual speech input.
    Implements few-shot prompting and structured output generation.
    """
    
    def __init__(self, model):
        """
        Initialize the email generator.
        
        Args:
            model: The Gemini model to use for generation
        """
        self.model = model
        self.email_history = []  # To maintain context across interactions
    
    def prepare_few_shot_examples(self):
        """
        Create few-shot examples to guide the email generation.
        
        Returns:
            str: Few-shot examples formatted as text
        """
        examples = [
            {
                "input": "I need to email John about the budget report that's due next week and ask if he needs any help gathering the Q3 numbers",
                "output": {
                    "to": "John",
                    "subject": "Q3 Budget Report - Assistance Offer",
                    "greeting": "Hi John,",
                    "body": "I wanted to touch base regarding the budget report due next week. I've cleared some time in my schedule and would be happy to assist with gathering the Q3 numbers if that would be helpful.\n\nPlease let me know if you'd like to collaborate on this, or if there's any specific section where you could use an extra hand.",
                    "closing": "Looking forward to your response,\n[Your Name]",
                    "tone": "professional, helpful"
                }
            },
            {
                "input": "Need to tell my team about postponing tomorrow's meeting to next Monday same time because I have a doctor's appointment",
                "output": {
                    "to": "Team",
                    "subject": "Tomorrow's Meeting Rescheduled to Monday",
                    "greeting": "Hello everyone,",
                    "body": "I need to reschedule our meeting originally planned for tomorrow to next Monday at the same time due to a medical appointment that cannot be changed.\n\nAll agenda items will remain the same, and I appreciate your flexibility with this change. Please let me know if you have any concerns about the new date and time.",
                    "closing": "Thank you for your understanding,\n[Your Name]",
                    "tone": "direct, appreciative"
                }
            }
        ]
        
        # Format examples as text
        examples_text = "# Few-shot examples for email generation\n\n"
        for i, example in enumerate(examples):
            examples_text += f"## Example {i+1}\n"
            examples_text += f"User says: \"{example['input']}\"\n\n"
            examples_text += "Generated email:\n"
            examples_text += f"TO: {example['output']['to']}\n"
            examples_text += f"SUBJECT: {example['output']['subject']}\n"
            examples_text += f"{example['output']['greeting']}\n\n"
            examples_text += f"{example['output']['body']}\n\n"
            examples_text += f"{example['output']['closing']}\n"
            examples_text += f"Tone: {example['output']['tone']}\n\n"
        
        return examples_text
    
    def generate_email(self, transcription):
        """
        Generate a professional email based on the transcribed speech.
        
        Args:
            transcription (str): Transcribed speech from the user
            
        Returns:
            dict: Structured email with components (to, subject, body, etc.)
        """
        # Prepare few-shot examples
        few_shot_examples = self.prepare_few_shot_examples()
        
        # Add historical context if available
        context = ""
        if self.email_history:
            context = "Previous emails in this conversation:\n\n"
            for i, email in enumerate(self.email_history[-2:]):  # Last 2 emails
                context += f"Email {i+1}:\n{email}\n\n"
        
        # Define the JSON format separately as a regular string (not an f-string)
        json_format = '''{
            "to": "recipient name or department",
            "subject": "clear and concise subject line",
            "greeting": "appropriate greeting",
            "body": "professional email body with paragraphs separated by \\n\\n",
            "closing": "appropriate closing and signature placeholder",
            "tone": "2-3 adjectives describing the email's tone"
        }'''
        
        # Then include it in your f-string
        prompt = f"""
        {few_shot_examples}
        
        {context if context else ""}
        
        Based on the user's casual speech input below, generate a professional email.
        
        User's input: "{transcription}"
        
        Return your response in this specific JSON format:
        {json_format}
        
        Make sure the email is professional, grammatically correct, and captures all the key points from the user's input.
        The tone should match the user's intent while maintaining professionalism.
        """

        
        # Generate the email using Gemini with function calling for structured output
        generation_config = {
            "temperature": 0.7,
            "top_p": 0.95,
            "top_k": 64,
            "max_output_tokens": 1024,
        }
        
        # Make the API call
        response = self.model.generate_content(prompt, generation_config=generation_config)
        
        # Extract and parse the JSON response
        try:
            # Find JSON in the response
            content = response.text
            # Look for JSON pattern - anything between { and }
            import re
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                email_data = json.loads(json_str)
            else:
                # Fallback parsing
                email_data = {
                    "to": "Recipient",
                    "subject": "Subject",
                    "greeting": "Hello,",
                    "body": content,
                    "closing": "Regards,\n[Your Name]",
                    "tone": "professional"
                }
        except Exception as e:
            print(f"Error parsing response: {e}")
            # Fallback to a basic structure
            email_data = {
                "to": "Recipient",
                "subject": "Subject",
                "greeting": "Hello,",
                "body": response.text,
                "closing": "Regards,\n[Your Name]",
                "tone": "professional"
            }
        
        # Format the full email for display and history
        formatted_email = f"""
        TO: {email_data['to']}
        SUBJECT: {email_data['subject']}
        
        {email_data['greeting']}
        
        {email_data['body']}
        
        {email_data['closing']}
        """
        
        # Save to history for context in future generations
        self.email_history.append(formatted_email)
        
        return email_data
    
    def format_email_html(self, email_data):
        """
        Format the email as HTML for display.
    
        Args:
            email_data (dict): The structured email data
    
        Returns:
            str: HTML formatted email
        """
        # Perform replacements outside the f-string
        body_html = email_data['body'].replace('\n\n', '</p><p>')
        closing_html = email_data['closing'].replace('\n', '<br>')
    
        html = f"""
        <div style="font-family: Arial, sans-serif; max-width: 600px; margin: 20px auto; padding: 20px; border: 1px solid #ccc; border-radius: 5px;">
            <div style="margin-bottom: 20px;">
                <div><strong>To:</strong> {email_data['to']}</div>
                <div><strong>Subject:</strong> {email_data['subject']}</div>
                <div><strong>Tone:</strong> <span style="font-style: italic;">{email_data['tone']}</span></div>
            </div>
            <div style="padding: 15px; background-color: #f9f9f9; border-radius: 5px;">
                <p>{email_data['greeting']}</p>
                <div style="margin: 15px 0;">
                    {body_html}
                </div>
                <p>{closing_html}</p>
            </div>
            <div style="margin-top: 20px; font-size: 0.8em; color: #666;">
                <p>⚠️ Don't forget to check if you need to add any attachments!</p>
            </div>
        </div>
        """
        return html




## Email Evaluator Module
class EmailEvaluator:
    """
    Evaluates the quality of generated emails using the Gemini API.
    Provides feedback on professionalism, completeness, and tone.
    """
    
    def __init__(self, model):
        """Initialize the evaluator."""
        self.model = model
    
    def evaluate_email(self, original_input, generated_email):
        """Evaluate the quality of a generated email."""
        # Define the JSON format separately
        eval_json_format = '''{
            "completeness_score": score,
            "professionalism_score": score,
            "clarity_score": score,
            "grammar_score": score,
            "overall_score": average_score,
            "feedback": "specific improvement suggestions"
        }'''
        
        # Construct prompt for evaluation
        prompt = f"""
        Evaluate the quality of this AI-generated email based on the user's original input.
        
        Original user input: "{original_input}"
        
        Generated email:
        TO: {generated_email['to']}
        SUBJECT: {generated_email['subject']}
        {generated_email['greeting']}
        
        {generated_email['body']}
        
        {generated_email['closing']}
        
        Please evaluate on the following criteria and provide a score from 1-10 for each:
        1. Completeness: Does the email address all points mentioned in the user input?
        2. Professionalism: Is the language, tone, and structure appropriate for a business email?
        3. Clarity: Is the message clear and easy to understand?
        4. Grammar & Style: Is the email free of errors and well-written?
        
        Also provide brief feedback on how the email could be improved.
        
        Return your response in this JSON format:
        {eval_json_format}
        """
        
        # Generate the evaluation
        generation_config = {
            "temperature": 0.2,  # Lower temperature for more consistent evaluation
            "max_output_tokens": 1024,
        }
        
        response = self.model.generate_content(prompt, generation_config=generation_config)
        
        # Parse the JSON response
        try:
            # Extract JSON from response
            content = response.text
            import re
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                evaluation = json.loads(json_str)
            else:
                # Fallback
                evaluation = {
                    "completeness_score": 7,
                    "professionalism_score": 8,
                    "clarity_score": 7,
                    "grammar_score": 8,
                    "overall_score": 7.5,
                    "feedback": "The evaluation couldn't be properly parsed, but the email appears to be of good quality."
                }
        except Exception as e:
            print(f"Error parsing evaluation: {e}")
            evaluation = {
                "completeness_score": 7,
                "professionalism_score": 8,
                "clarity_score": 7,
                "grammar_score": 8,
                "overall_score": 7.5,
                "feedback": "An error occurred during evaluation."
            }
        
        return evaluation
    
    def format_evaluation_html(self, evaluation):
        """Format the evaluation as HTML for display."""
        # Create HTML as a regular string first, then format it
        html_template = '''
        <div style="font-family: Arial, sans-serif; max-width: 600px; margin: 20px auto; padding: 20px; border: 1px solid #ddd; border-radius: 5px; background-color: #f5f5f5;">
            <h3 style="margin-top: 0; color: #444;">Email Quality Evaluation</h3>
            
            <div style="display: flex; justify-content: space-between; margin-bottom: 15px;">
                <div style="flex: 1;">
                    <div style="font-weight: bold; margin-bottom: 5px;">Overall Score</div>
                    <div style="font-size: 24px; color: #0066cc;">{overall}/10</div>
                </div>
                <div style="flex: 3;">
                    <table style="width: 100%; border-collapse: collapse;">
                        <tr>
                            <td style="padding: 5px 10px;">Completeness:</td>
                            <td style="padding: 5px 10px;"><progress value="{completeness}" max="10" style="width: 100%;"></progress></td>
                            <td style="padding: 5px 10px; width: 30px; text-align: right;">{completeness}</td>
                        </tr>
                        <tr>
                            <td style="padding: 5px 10px;">Professionalism:</td>
                            <td style="padding: 5px 10px;"><progress value="{professionalism}" max="10" style="width: 100%;"></progress></td>
                            <td style="padding: 5px 10px; width: 30px; text-align: right;">{professionalism}</td>
                        </tr>
                        <tr>
                            <td style="padding: 5px 10px;">Clarity:</td>
                            <td style="padding: 5px 10px;"><progress value="{clarity}" max="10" style="width: 100%;"></progress></td>
                            <td style="padding: 5px 10px; width: 30px; text-align: right;">{clarity}</td>
                        </tr>
                        <tr>
                            <td style="padding: 5px 10px;">Grammar & Style:</td>
                            <td style="padding: 5px 10px;"><progress value="{grammar}" max="10" style="width: 100%;"></progress></td>
                            <td style="padding: 5px 10px; width: 30px; text-align: right;">{grammar}</td>
                        </tr>
                    </table>
                </div>
            </div>
            
            <div style="padding: 15px; background-color: white; border-radius: 5px; border-left: 4px solid #0066cc;">
                <strong>Feedback:</strong>
                <p style="margin-top: 5px;">{feedback}</p>
            </div>
        </div>
        '''
        
        # Now format the HTML with the evaluation data
        html = html_template.format(
            overall=evaluation['overall_score'],
            completeness=evaluation['completeness_score'],
            professionalism=evaluation['professionalism_score'],
            clarity=evaluation['clarity_score'],
            grammar=evaluation['grammar_score'],
            feedback=evaluation['feedback']
        )
        
        return html

**Demonstration Functions**

In [3]:
## Demonstration Functions
def demonstrate_workflow():
    """Run a complete demonstration of the Panda Email Pro workflow."""
    
    print("🐼 Welcome to Panda Email Pro Demonstration 🐼")
    print("-" * 50)
    
    # Initialize components
    audio_processor = AudioProcessor()
    email_generator = EmailGenerator(model)
    email_evaluator = EmailEvaluator(model)
    
    # Step 1: Record audio input
    print("Step 1: Processing audio input...")
    print("Please speak your email request when prompted.")
    
    try:
        # Try to record actual audio
        audio_file_path = audio_processor.record_audio(duration=15)
        transcription = audio_processor.process_audio_file(audio_file_path)
    except Exception as e:
        print(f"Error recording audio: {e}")
        # Fall back to simulated transcription
        transcription = """
        Hey, I need to send an email to Dr. Rodriguez from the research department
        about our latest experiment results. We found that the new compound is showing
        85% efficacy in lab tests, which is really promising. I want to suggest we move
        to phase 2 testing next month if he agrees with our analysis. Also, mention that
        the full dataset is ready for review and I can present findings at the next team meeting.
        Keep it professional but show my excitement about these results.
        """
        print("Using simulated transcription instead.")
    
    print("\nTranscribed Audio:")
    print(f"\"{transcription.strip()}\"")
    print("-" * 50)
    
    # Step 2: Generate the email
    print("Step 2: Generating professional email...")
    email_data = email_generator.generate_email(transcription)
    
    # Display the generated email
    print("\nGenerated Email:")
    print(f"TO: {email_data['to']}")
    print(f"SUBJECT: {email_data['subject']}")
    print(f"\n{email_data['greeting']}")
    print(f"\n{email_data['body']}")
    print(f"\n{email_data['closing']}")
    print(f"\nTone: {email_data['tone']}")
    print("-" * 50)
    
    # Step 3: Evaluate the email
    print("Step 3: Evaluating email quality...")
    evaluation = email_evaluator.evaluate_email(transcription, email_data)
    
    print("\nEmail Evaluation:")
    print(f"Overall Score: {evaluation['overall_score']}/10")
    print(f"Completeness: {evaluation['completeness_score']}/10")
    print(f"Professionalism: {evaluation['professionalism_score']}/10")
    print(f"Clarity: {evaluation['clarity_score']}/10")
    print(f"Grammar & Style: {evaluation['grammar_score']}/10")
    print(f"\nFeedback: {evaluation['feedback']}")
    print("-" * 50)
    
    # Step 4: Display formatted HTML version
    print("Step 4: Displaying formatted email...")
    html_email = email_generator.format_email_html(email_data)
    display(HTML(html_email))
    
    html_evaluation = email_evaluator.format_evaluation_html(evaluation)
    display(HTML(html_evaluation))
    
    print("\n✅ Demonstration complete!")
    print("In a real application, the user would now have the option to:")
    print("- Edit the generated email")
    print("- Attach files (with a reminder already included)")
    print("- Send the email directly")
    print("- Save as a draft")

def create_gradio_interface():
    """Create a simple Gradio interface for testing the application."""
    
    audio_processor = AudioProcessor()
    email_generator = EmailGenerator(model)
    email_evaluator = EmailEvaluator(model)
    
    def process_audio(audio_file):
        """Process audio file and generate email."""
        if audio_file is not None:
            # Save the recorded audio to a temporary file
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.wav')
            temp_file.close()
            
            # Save the audio data to the temp file
            sf.write(temp_file.name, audio_file[1], audio_file[0])
            
            # Process the audio to get transcription
            transcription = audio_processor.process_audio_file(temp_file.name)
        else:
            # Fallback example
            transcription = """
            I need to email the team about the project deadline extension. We got approval 
            to move it from June 15 to June 30. We should explain that this will allow us 
            to add the new features the client requested last week. Also mention that I'll 
            need everyone's updated timelines by this Friday.
            """
        
        # Generate email
        email_data = email_generator.generate_email(transcription)
        
        # Create formatted version
        formatted_email = f"""
        TO: {email_data['to']}
        SUBJECT: {email_data['subject']}
        
        {email_data['greeting']}
        
        {email_data['body']}
        
        {email_data['closing']}
        
        Tone: {email_data['tone']}
        """
        
        # Evaluate
        evaluation = email_evaluator.evaluate_email(transcription, email_data)
        eval_summary = f"""
        Overall Score: {evaluation['overall_score']}/10
        Completeness: {evaluation['completeness_score']}/10
        Professionalism: {evaluation['professionalism_score']}/10
        Clarity: {evaluation['clarity_score']}/10
        Grammar: {evaluation['grammar_score']}/10
        
        Feedback: {evaluation['feedback']}
        """
        
        return transcription, formatted_email, eval_summary
    
    # Create text input version for easier testing
    def process_text(text_input):
        """Process text input instead of audio."""
        # Generate email
        email_data = email_generator.generate_email(text_input)
        
        # Create formatted version
        formatted_email = f"""
        TO: {email_data['to']}
        SUBJECT: {email_data['subject']}
        
        {email_data['greeting']}
        
        {email_data['body']}
        
        {email_data['closing']}
        
        Tone: {email_data['tone']}
        """
        
        # Evaluate
        evaluation = email_evaluator.evaluate_email(text_input, email_data)
        eval_summary = f"""
        Overall Score: {evaluation['overall_score']}/10
        Completeness: {evaluation['completeness_score']}/10
        Professionalism: {evaluation['professionalism_score']}/10
        Clarity: {evaluation['clarity_score']}/10
        Grammar: {evaluation['grammar_score']}/10
        
        Feedback: {evaluation['feedback']}
        """
        
        return text_input, formatted_email, eval_summary
    
    # Create interface with tabs for audio or text input
    with gr.Blocks(title="Panda Email Pro") as interface:
        gr.Markdown("# 🐼 Panda Email Pro")
        gr.Markdown("Convert casual speech to professional emails using AI")
        
        with gr.Tabs():
            with gr.TabItem("Audio Input"):
                audio_input = gr.Audio(label="Record your email request", type="numpy", sources=["microphone"])
                audio_button = gr.Button("Process Audio & Generate Email")
                
                audio_transcription = gr.Textbox(label="Transcribed Audio", lines=3)
                audio_email = gr.Textbox(label="Generated Email", lines=10)
                audio_evaluation = gr.Textbox(label="Email Evaluation", lines=8)
                
                audio_button.click(process_audio, inputs=[audio_input], outputs=[audio_transcription, audio_email, audio_evaluation])
                
            with gr.TabItem("Text Input (For Testing)"):
                text_input = gr.Textbox(
                    label="Describe what you want in your email",
                    placeholder="Example: I need to email my boss about taking next Friday off because I have a doctor's appointment...",
                    lines=5
                )
                text_button = gr.Button("Generate Email")
                
                text_transcription = gr.Textbox(label="Your Input", lines=3)
                text_email = gr.Textbox(label="Generated Email", lines=10)
                text_evaluation = gr.Textbox(label="Email Evaluation", lines=8)
                
                text_button.click(process_text, inputs=[text_input], outputs=[text_transcription, text_email, text_evaluation])
                
        gr.Markdown("## How to use Panda Email Pro")
        gr.Markdown("""
        1. **Speak into your microphone** about the email you want to send
        2. Specify **who** you're emailing and **what** you want to say
        3. Mention any **tone preferences** (formal, friendly, urgent, etc.)
        4. The AI will generate a professional email draft
        5. Review, edit if needed, and don't forget attachments!
        """)
        
    return interface

**Run Demonstration and Launch Interface**

In [4]:
# Run the demonstration when the notebook is executed
print("Starting Panda Email Pro demonstration...")
demonstrate_workflow()

# Launch the Gradio interface
interface = create_gradio_interface()
interface.launch()

Starting Panda Email Pro demonstration...
🐼 Welcome to Panda Email Pro Demonstration 🐼
--------------------------------------------------
Step 1: Processing audio input...
Please speak your email request when prompted.
Recording audio for 15 seconds...
Error recording audio: Could not find PyAudio; check installation
Created silent audio file instead.
Google Speech Recognition could not understand audio

Transcribed Audio:
"I couldn't understand the audio. Please try again."
--------------------------------------------------
Step 2: Generating professional email...

Generated Email:
TO: Sender of the Audio
SUBJECT: Audio Unintelligible - Request to Resend

Dear [Sender Name],

I am writing to inform you that I was unable to understand the audio you sent.

Could you please resend the audio file?  If possible, please check the audio quality before resending to ensure it is clear.

Thank you for your assistance.

Sincerely,
[Your Name]

Tone: polite, direct, professional
-----------------

Completeness:,,10
Professionalism:,,9
Clarity:,,10
Grammar & Style:,,10



✅ Demonstration complete!
In a real application, the user would now have the option to:
- Edit the generated email
- Attach files (with a reminder already included)
- Send the email directly
- Save as a draft
* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://05a80fc58d11e1546e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Conclusion and Technical Notes**

In [5]:
## Conclusion and Future Work
# Panda Email Pro demonstrates the power of combining audio processing with generative AI
# to streamline professional communication. The project successfully implements multiple
# Gen AI capabilities including:
#
# 1. Audio Understanding - Processing spoken language into actionable content
# 2. Structured Output/JSON Mode - Ensuring consistent email formatting
# 3. Few-shot Prompting - Improving email quality through examples
# 4. Long Context Window - Maintaining conversation history
# 5. Gen AI Evaluation - Assessing email quality objectively
#
# Future enhancements could include:
# - Integration with email clients for direct sending
# - Multilingual support for global teams
# - Custom templates for different email types (status updates, requests, etc.)
# - Voice-based editing capabilities
# - Attachment suggestion based on email content

## Technical Implementation Notes
# This implementation uses the Gemini API from Google for its advanced natural language
# capabilities. The project structure follows modular design principles, separating concerns into:
#
# - Audio processing (capturing and transcribing speech)
# - Email generation (creating professional content from casual input)
# - Evaluation (assessing quality and suggesting improvements)
# - User interface (making the tool accessible)
#
# The few-shot prompting approach significantly improves output quality by providing the model
# with clear examples of the desired format and style.
#
# Performance optimization considerations include:
# - Caching previous outputs to reduce API calls
# - Streaming responses for better user experience
# - Handling edge cases gracefully with fallback options